**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [42]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [43]:
df_ori_dest = pd.read_csv('./dados/origem_porto.csv')
df_ori_trans = pd.read_csv('./dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('./dados/transbordo_porto.csv')

In [44]:
df_supply = pd.read_csv('./dados/supply.csv')
df_cap_port = pd.read_csv('./dados/cap_porto.csv')
df_cap_trans = pd.read_csv('./dados/cap_transbordo.csv')
df_demand = pd.read_csv('./dados/demand.csv')

In [45]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [46]:
df_supply

,0
0,479
1,295
2,621
3,789
4,465
5,530
6,891
7,860
8,630
9,733


In [47]:
df_demand

,0
0,2056
1,2149
2,2088


In [48]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

ways = {}
for i in M:
    ways[i] = {k for k in K}

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j, 0] = df_ori_dest[str(j - M[0])][i]
        for k in K:
            cost[i, j, k] = df_ori_trans[str(k - K[0])][i] + df_trans_porto[str(j - M[0])][k - K[0]]

K.append(0)

In [49]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

cap_transbordo[0] = max(demanda_total, oferta_total)
print(oferta_total, demanda_total)

6293 6293


In [50]:
m = gp.Model("probrema")

In [51]:
X = {}
for i in N:
    for j in M:
        for k in K:
            if not k in ways[j]: continue
            for o in O:
                X[i, j, k, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}_{}_{}".format(i, j, k, o))

In [52]:
m.setObjective(
    gp.quicksum((X[i, j, k, o] * cost[i, j, k]) for i in N for j in M for k in K for o in O if k in ways[j]),
    sense=gp.GRB.MINIMIZE
)

In [53]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for j in M for k in K for o in O if k in ways[j]) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for j in M for k in K for o in O if k in ways[j]) <= supply[i]
        )

In [54]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) == demand[o]
        )

In [55]:
for j in M:
    m.addConstr(
        gp.quicksum(X[i, j, k, o] for i in N for k in K for o in O if k in ways[j]) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, j, k, o] for i in N for j in M for o in O if k in ways[j]) <= cap_transbordo[k]
    )

In [56]:
m.addConstrs(
    gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) >= 0 for o in O
)

{21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>}

In [57]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 840 columns and 4200 nonzeros
Model fingerprint: 0x46b3bb15
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+03]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 24 rows, 840 columns, 3360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7504199e+05   2.253537e+03   0.000000e+00      0s
      72    2.6048566e+05   0.000000e+00   0.000000e+00      0s

Solved in 72 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.604856567e+05


In [58]:
m.write("out.sol")